In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os import getcwd
from scipy.constants import e, k

inp_dir = getcwd() + '/input_files/'

temp = 25
T = temp + 273
v_t = k * T / e

In [ ]:
data = read_file(inp_dir + 'diode_iv_curve.txt')
v = np.array([val for i, val in enumerate(data) if not i % 2])
i = np.array([val for i, val in enumerate(data) if i % 2])

write_file(inp_dir + 'diode_i_curve.txt', i)
write_file(inp_dir + 'diode_u_curve.txt', v)

In [ ]:
zero_cur = init_cur = i[0] / np.exp(v[0] / v_t - 1)
target_error = 0.1
errors = []


while True:
    eta, error = minimize(init_cur)
    init_cur = i[0] / (np.exp(v[0] / v_t / eta) - 1)
    metric = abs(init_cur - zero_cur) / zero_cur * 100
    # print(init_cur, zero_cur, metric)
    
    
    if metric < target_error:
        break
        
print(init_cur, eta)

In [ ]:
steps = 100
min_error = 10 ** 10
x_fit_vals = v[-12:]
y_fit_vals = i[-12:]

init_cur = i[0] / (np.exp(v[0] / v_t) - 1)
fit = np.polyfit(x_fit_vals, y_fit_vals, 1)
init_rs = 1 / fit[0]


for idx, eta in enumerate(np.linspace(1, 2, steps)):
    print_progress_bar(idx + 1, steps)
    for cur in np.linspace(0.75 * init_cur, 1.25 * init_cur, steps):
        for rs in np.linspace(0.75 * init_rs, 1.25 * init_rs, steps):
            approx = cur * (np.exp((v - rs * i) / v_t / eta) - 1)
            error = rmse(i, approx)
            errors.append(error)
    
            if error < min_error:
                min_error = error
                min_eta, min_cur, min_rs = eta, cur, rs
                
print(min_eta, min_cur, min_rs)

In [ ]:
eps = 10e-2
eta = 0.1

a1 = min_cur * (np.exp((v - min_rs * i) / v_t / min_eta) - 1)

ii = [0]
vv = np.linspace(0, v[-1], 100)

for idx, v_val in enumerate(vv[1:]):    
    old_c = ii[-1]
    new_c = min_cur * (np.exp((v_val - min_rs * old_c) / v_t / min_eta) - 1)
    
    while abs(new_c - old_c) > eps:
        old_c = new_c
        new_c = min_cur * (np.exp((v_val - min_rs * old_c) / v_t / min_eta) - 1)
        new_c = old_c + eta * (new_c - old_c)
    
    ii.append(new_c)


plt.plot(-vv, ii)
plt.plot(v, i)
plt.xlabel('Voltage (V)')
plt.ylabel('Current (A)')
plt.title('I-V Curve of Diode')
plt.grid(True)
plt.show()

In [ ]:
solar_cell_num = 54
G = 800

r_file = open("C:\\Users\\Martyniuk Vadym\\Desktop\\KC200GT_new.txt", "r")
e_voltage, e_current = get_data(r_file)
e_voltage = np.array(e_voltage) / solar_cell_num
r_file.close()

My_one_diode_fa_KC200GT = SolarCell(cell_num=54/ solar_cell_num, Voc=32.9 / solar_cell_num, Isc=8.21,
                       #Pmpp=200, Impp=7.61, Vmpp=26.3,
                       #Ku=-0.123, Ki=3.18 / 1000, a1=1.0, a2=10**10,
                       thermal_coefficient_type='absolute', G=800,
                       experimental_V=e_voltage,
                       experimental_I=e_current,
                       is_experimental_data_provided=True,
                       mode='overall')

My_one_diode_fa_KC200GT.end_fit_points_number

In [ ]:
My_one_diode_fa_KC200GT.run()

In [ ]:
v_rev = -vv
i_rev = My_one_diode_fa_KC200GT.fixed_point_method_rv(My_one_diode_fa_KC200GT.Rs, 
                                                      My_one_diode_fa_KC200GT.Rp, v_rev)[1:]
i_rev_diode = np.sum([i_rev, ii], axis=0)


i_for = My_one_diode_fa_KC200GT.fixed_point_method_rv(My_one_diode_fa_KC200GT.Rs, 
                                                      My_one_diode_fa_KC200GT.Rp, My_one_diode_fa_KC200GT.V_mesh)[1:]

In [ ]:
aa = np.array(My_one_diode_fa_KC200GT.V_mesh) * 300 / 1000

In [ ]:
plt.plot(My_one_diode_fa_KC200GT.e_V, My_one_diode_fa_KC200GT.e_I)
plt.plot(aa, i_for)
plt.plot(v_rev, i_rev)
plt.plot(v_rev, i_rev_diode)
plt.grid()

In [ ]:
KC200GT_1000 = SolarCell(cell_num=54/ solar_cell_num, Voc=32.9 / solar_cell_num, Isc=8.21,
                         thermal_coefficient_type='absolute', G=1000,
                         experimental_V=e_voltage,
                         experimental_I=e_current,
                         is_experimental_data_provided=True,
                         mode='overall')

KC200GT_1000.run()

In [ ]:
KC200GT_800 = SolarCell(cell_num=54/ solar_cell_num, Voc=32.9 / solar_cell_num, Isc=8.21,
                         thermal_coefficient_type='absolute', G=600,
                         experimental_V=e_voltage,
                         experimental_I=e_current,
                         is_experimental_data_provided=True,
                         mode='overall')

KC200GT_800.run()

In [ ]:
plt.plot(My_one_diode_fa_KC200GT.V_mesh, My_one_diode_fa_KC200GT.current)
plt.plot(voltage800, current800[1:])